In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [2]:
with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 146s 48ms/step - loss: 6.2292 - accuracy: 0.0767
Epoch 2/100
3010/3010 [==============================] - 141s 47ms/step - loss: 5.4952 - accuracy: 0.1257
Epoch 3/100
3010/3010 [==============================] - 144s 48ms/step - loss: 5.1122 - accuracy: 0.1481
Epoch 4/100
3010/3010 [==============================] - 144s 48ms/step - loss: 4.7855 - accuracy: 0.1659
Epoch 5/100
3010/3010 [==============================] - 141s 47ms/step - loss: 4.4877 - accuracy: 0.1824
Epoch 6/100
3010/3010 [==============================] - 144s 48ms/step - loss: 4.2045 - accuracy: 0.2016
Epoch 7/100
3010/3010 [==============================] - 143s 47ms/step - loss: 3.9320 - accuracy: 0.2291
Epoch 8/100
3010/3010 [==============================] - 143s 48ms/step - loss: 3.6741 - accuracy: 0.2583
Epoch 9/100
3010/3010 [==============================] - 143s 47ms/step - loss: 3.4322 - accuracy: 0.2912
Epoch 10/100
3010/3010 [======================

3010/3010 [==============================] - 124s 41ms/step - loss: 0.5467 - accuracy: 0.8615
Epoch 78/100
3010/3010 [==============================] - 121s 40ms/step - loss: 0.5429 - accuracy: 0.8613
Epoch 79/100
3010/3010 [==============================] - 123s 41ms/step - loss: 0.5431 - accuracy: 0.8607
Epoch 80/100
3010/3010 [==============================] - 125s 42ms/step - loss: 0.5391 - accuracy: 0.8627
Epoch 81/100
3010/3010 [==============================] - 124s 41ms/step - loss: 0.5428 - accuracy: 0.8600
Epoch 82/100
3010/3010 [==============================] - 124s 41ms/step - loss: 0.5340 - accuracy: 0.8631
Epoch 83/100
3010/3010 [==============================] - 125s 42ms/step - loss: 0.5419 - accuracy: 0.8596
Epoch 84/100
3010/3010 [==============================] - 121s 40ms/step - loss: 0.5332 - accuracy: 0.8627
Epoch 85/100
3010/3010 [==============================] - 125s 41ms/step - loss: 0.5283 - accuracy: 0.8649
Epoch 86/100
3010/3010 [==========================

In [10]:
seed_text = "speed"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 47ms/step
speed down home in


In [11]:
import pickle

# Save the next word prediction model to a file
f = open('next_word_prediction_model.pickle', 'wb')
pickle.dump(model, f)
print('Classifier stored at ', f.name)
f.close()

Classifier stored at  next_word_prediction_model.pickle
